# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,omdurman,15.6445,32.4777,30.06,9,4,3.59,SD,1714014226
1,1,kone,-21.0595,164.8658,27.89,61,22,4.49,NC,1714014227
2,2,igarka,67.4667,86.5833,-7.21,90,79,3.58,RU,1714014228
3,3,avarua,-21.2078,-159.7750,26.03,73,75,3.09,CK,1714014229
4,4,bonito,-21.1211,-56.4819,24.30,75,34,1.35,BR,1714014043


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_height = 500,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"]< 27) & (city_data_df["Max Temp"]> 21) & (city_data_df["Wind Speed"]<4.5) &(city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,wailea,19.8858,-155.1230,25.57,82,0,3.60,US,1714014305
169,169,ghat,24.9647,10.1728,21.39,19,0,2.95,LY,1714014392
218,218,al jahra',29.3375,47.6581,26.46,31,0,2.27,KW,1714014442
248,248,lazaro cardenas,17.9583,-102.2000,24.86,79,0,1.54,MX,1714014179
266,266,pisco,-13.7000,-76.2167,21.03,78,0,2.06,PE,1714014229
295,295,el rama,12.1596,-84.2195,24.03,89,0,1.49,NI,1714014515
347,347,piton saint-leu,-21.2196,55.3151,22.04,78,0,1.54,RE,1714014572
361,361,bayji,34.9311,43.4870,23.88,24,0,3.27,IQ,1714014589
365,365,prado,-17.3411,-39.2208,25.21,85,0,2.36,BR,1714014403
388,388,saint-pierre,-21.3393,55.4781,22.82,78,0,1.54,RE,1714014573


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
column = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = city_data_df[column].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
82,wailea,US,19.8858,-155.1230,82,
169,ghat,LY,24.9647,10.1728,19,
218,al jahra',KW,29.3375,47.6581,31,
248,lazaro cardenas,MX,17.9583,-102.2000,79,
266,pisco,PE,-13.7000,-76.2167,78,
295,el rama,NI,12.1596,-84.2195,89,
347,piton saint-leu,RE,-21.2196,55.3151,78,
361,bayji,IQ,34.9311,43.4870,24,
365,prado,BR,-17.3411,-39.2208,85,
388,saint-pierre,RE,-21.3393,55.4781,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "format": "json",
    "limit": 5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    long = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailea - nearest hotel: Hakalau Vacation Rental
ghat - nearest hotel: فندق تاسيلي
al jahra' - nearest hotel: Al Jahra Copthorne Hotel & Resort
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
pisco - nearest hotel: La Portada
el rama - nearest hotel: Hotel Oasis
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
bayji - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
saint-pierre - nearest hotel: Tropic Hotel
yuanping - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
82,wailea,US,19.8858,-155.1230,82,Hakalau Vacation Rental
169,ghat,LY,24.9647,10.1728,19,فندق تاسيلي
218,al jahra',KW,29.3375,47.6581,31,Al Jahra Copthorne Hotel & Resort
248,lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
266,pisco,PE,-13.7000,-76.2167,78,La Portada
295,el rama,NI,12.1596,-84.2195,89,Hotel Oasis
347,piton saint-leu,RE,-21.2196,55.3151,78,Appartement Piton Saint Leu
361,bayji,IQ,34.9311,43.4870,24,No hotel found
365,prado,BR,-17.3411,-39.2208,85,Pousada Recanto do Prado
388,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plt= hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_height = 500, 
    frame_width = 700,
    hover_cols = "Hotel Name",
    size = "Humidity",
    color = "City")

# Display the map
hotel_plt

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)